In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 25 04:09:59 2020

@author: ASUS
"""

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import datetime


def image_gen_w_aug(train_parent_directory, valid_test_parent_directory, test_parent_directory):
    train_datagen = ImageDataGenerator(rescale=1 / 255,
                                       rotation_range=30,
                                       zoom_range=0.2,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1)

    test_datagen = ImageDataGenerator(rescale=1 / 255)

    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                        target_size=(75, 75),
                                                        batch_size=37,
                                                        class_mode='categorical')

    val_generator = train_datagen.flow_from_directory(valid_test_parent_directory,
                                                      target_size=(75, 75),
                                                      batch_size=37,
                                                      class_mode='categorical')

    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                      target_size=(75, 75),
                                                      batch_size=37,
                                                      class_mode='categorical')

    return train_generator, val_generator, test_generator


def model_output_for_TL(pre_trained_model, last_output):
    x = Flatten()(last_output)

    # Dense hidden layer
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

    # Output neuron. 
    x = Dense(3, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)

    return model


train_dir = os.path.join('C:/Python/Apple-Lemon-Detector/datasets/train/')
test_dir = os.path.join('C:/Python/Apple-Lemon-Detector/datasets/test/')
val_dir = os.path.join('C:/Python/Apple-Lemon-Detector/datasets/val/')

train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, val_dir, test_dir)

pre_trained_model = InceptionV3(input_shape=(75, 75, 3),
                                include_top=False,
                                weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed3')
last_output = last_layer.output

model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

log_dir = "logs/inception/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch='100,120')
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                 profile_batch='500,510')

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    profile_batch='100,120')

history = model_TL.fit(
    train_generator,
    steps_per_epoch=10,  # 10 batches to train
    epochs=5,
    verbose=1,
    validation_data=validation_generator,
    callbacks=[tboard_callback])

# tf.keras.models.save_model(model_TL, 'my_model.hdf5')

Found 1870 images belonging to 3 classes.
Found 233 images belonging to 3 classes.
Found 235 images belonging to 3 classes.
Epoch 1/5
10/10 [==============================] - 12s 388ms/step - loss: 11.6333 - acc: 0.5836 - val_loss: 3.4965 - val_acc: 0.6996
Epoch 2/5
10/10 [==============================] - 3s 273ms/step - loss: 2.1057 - acc: 0.7216 - val_loss: 0.1588 - val_acc: 0.9356
Epoch 3/5
10/10 [==============================] - 3s 275ms/step - loss: 0.7344 - acc: 0.8784 - val_loss: 0.4276 - val_acc: 0.9013
Epoch 4/5
10/10 [==============================] - 3s 256ms/step - loss: 0.1763 - acc: 0.9243 - val_loss: 0.0954 - val_acc: 0.9528
Epoch 5/5
10/10 [==============================] - 3s 254ms/step - loss: 0.1095 - acc: 0.9784 - val_loss: 0.0487 - val_acc: 0.9828
